## "3 types of Filterings (for recommendation systems)"
#### 1. Content based or item Based  : 
            In this algorithm we try finding item look alike . Once we have item look alike matrix ,we can easily recommend alike items to customer who has purchased any item from the store . (let say amazon flipkart fro selling similar products)
#### 2. Collaborative Filtering :
            Here , we try to search for look alike customers and offer products based n what his/her look alike has chosen . This algorithm is very effective but takes a lot of time and recources.(e.g. movies , books etc.)
#### 3. Hybrid :
            combinations of both (some of industries or saying when we need the combination of both)
            

### on kaggle dataset of books

In [4]:
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings(action = 'ignore') #ignoring the warnings through out this model building task

In [5]:
# pipe-line for load data from various extension and renaming the col. in lowercase also removing and replacing 'spaces' and '-' with '_' 
def load_dataset(file_path, **kwargs):
    """
    Load data from various file formats while handling common issues.
    
    Args:
        file_path (str): Path to the data file
        **kwargs: Additional arguments to pass to the appropriate pandas reader
    
    Returns:
        pd.DataFrame: Loaded and initially processed dataframe
    """
    import pandas as pd
    from pathlib import Path
    
    file_type = Path(file_path).suffix.lower()
    
    # Dictionary of file handlers
    handlers = {
        '.csv': pd.read_csv,
        '.xlsx': pd.read_excel,
        '.json': pd.read_json,
        '.parquet': pd.read_parquet
    }
    
    # Get appropriate reader function
    reader = handlers[file_type]
    if reader is None:
        raise ValueError(f"Unsupported file type: {file_type}")
    
    # Load data with common cleaning parameters
    df = reader(file_path, **kwargs)
    
    # Initial cleaning steps
    df.columns = df.columns.str.strip().str.lower().str.replace('-','_')  # Standardize column names
    df = df.replace('', pd.NA)  # Convert empty strings to NA
    
    return df

## Load Books Dataset

In [7]:
books = load_dataset('Books.csv')
books.head(3)

,isbn,book_title,book_author,year_of_publication,publisher,image_url_s,image_url_m,image_url_l
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...


In [8]:
books.isna().sum()

isbn                   0
book_title             0
book_author            2
year_of_publication    0
publisher              2
image_url_s            0
image_url_m            0
image_url_l            3
dtype: int64

In [9]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   isbn                 271360 non-null  object
 1   book_title           271360 non-null  object
 2   book_author          271358 non-null  object
 3   year_of_publication  271360 non-null  object
 4   publisher            271358 non-null  object
 5   image_url_s          271360 non-null  object
 6   image_url_m          271360 non-null  object
 7   image_url_l          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [10]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   isbn                 271360 non-null  object
 1   book_title           271360 non-null  object
 2   book_author          271358 non-null  object
 3   year_of_publication  271360 non-null  object
 4   publisher            271358 non-null  object
 5   image_url_s          271360 non-null  object
 6   image_url_m          271360 non-null  object
 7   image_url_l          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


## Load User Dataset

In [12]:
users = load_dataset('Users.csv')

In [13]:
users.head(3)

,user_id,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN


## Load Ratings Dataset

In [15]:
ratings = load_dataset('Ratings.csv')

In [16]:
ratings.head(2)

,user_id,isbn,book_rating
0,276725,034545104X,0
1,276726,0155061224,5


In [17]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   user_id      1149780 non-null  int64 
 1   isbn         1149780 non-null  object
 2   book_rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


## Data Cleaning & Feature Eng. :
## where we will took only those records which are 'more' meaningful. 
# ↓

In [19]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   isbn                 271360 non-null  object
 1   book_title           271360 non-null  object
 2   book_author          271358 non-null  object
 3   year_of_publication  271360 non-null  object
 4   publisher            271358 non-null  object
 5   image_url_s          271360 non-null  object
 6   image_url_m          271360 non-null  object
 7   image_url_l          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [20]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   user_id   278858 non-null  int64  
 1   location  278858 non-null  object 
 2   age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [21]:
# take those users as actual_readers who raetes more than at least 200 books .
ratings.user_id.value_counts()

user_id
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: count, Length: 105283, dtype: int64

In [22]:
ratings.user_id.value_counts().shape

(105283,)

In [23]:
actual_readers = ratings['user_id'].value_counts() > 200

In [24]:
actual_readers[actual_readers].shape

(899,)

In [25]:
actual_readers_id = actual_readers[actual_readers].index # extractings only user_id's which are as index in readers

In [26]:
actual_readers_id

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727, 268622,
       188951],
      dtype='int64', name='user_id', length=899)

In [27]:
ratings = ratings[ratings.user_id.isin(actual_readers_id)] #filtering ids with only actual readers 

In [28]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 526356 entries, 1456 to 1147616
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   user_id      526356 non-null  int64 
 1   isbn         526356 non-null  object
 2   book_rating  526356 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 16.1+ MB


In [29]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   isbn                 271360 non-null  object
 1   book_title           271360 non-null  object
 2   book_author          271358 non-null  object
 3   year_of_publication  271360 non-null  object
 4   publisher            271358 non-null  object
 5   image_url_s          271360 non-null  object
 6   image_url_m          271360 non-null  object
 7   image_url_l          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [30]:
books_with_ratings= books.merge(ratings , on = 'isbn') #International Standard Book Number : isbn → unique no. for every books 

In [31]:
books_with_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 487671 entries, 0 to 487670
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   isbn                 487671 non-null  object
 1   book_title           487671 non-null  object
 2   book_author          487670 non-null  object
 3   year_of_publication  487671 non-null  object
 4   publisher            487669 non-null  object
 5   image_url_s          487671 non-null  object
 6   image_url_m          487671 non-null  object
 7   image_url_l          487668 non-null  object
 8   user_id              487671 non-null  int64 
 9   book_rating          487671 non-null  int64 
dtypes: int64(2), object(8)
memory usage: 37.2+ MB


In [32]:
ratings.isbn.isin(books.isbn).value_counts() 

isbn
True     487671
False     38685
Name: count, dtype: int64

In [33]:
books_with_ratings.isbn.nunique()

178018

In [34]:
books_with_ratings.head(3)

,isbn,book_title,book_author,year_of_publication,publisher,image_url_s,image_url_m,image_url_l,user_id,book_rating
0,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,8
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,85526,0
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,96054,0


In [35]:
number_of_ratings = books_with_ratings.groupby('book_title')['book_rating'].count().sort_values(ascending = False).reset_index()

In [36]:
number_of_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160269 entries, 0 to 160268
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   book_title   160269 non-null  object
 1   book_rating  160269 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [37]:
number_of_ratings.rename(columns= {'book_rating' : 'number_of_ratings'} , inplace = True)

In [38]:
# merge it to the books_with_ratings to get the final result
final_ratings = books_with_ratings.merge(number_of_ratings , on = 'book_title')

In [39]:
final_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 487671 entries, 0 to 487670
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   isbn                 487671 non-null  object
 1   book_title           487671 non-null  object
 2   book_author          487670 non-null  object
 3   year_of_publication  487671 non-null  object
 4   publisher            487669 non-null  object
 5   image_url_s          487671 non-null  object
 6   image_url_m          487671 non-null  object
 7   image_url_l          487668 non-null  object
 8   user_id              487671 non-null  int64 
 9   book_rating          487671 non-null  int64 
 10  number_of_ratings    487671 non-null  int64 
dtypes: int64(3), object(8)
memory usage: 40.9+ MB


In [40]:
final_ratings['number_of_ratings'].sort_values(ascending=False)

227       363
285       363
311       363
310       363
309       363
         ... 
414150      1
414149      1
414147      1
414146      1
487670      1
Name: number_of_ratings, Length: 487671, dtype: int64

In [41]:
final_ratings = final_ratings[final_ratings['number_of_ratings'] >= 50] # consider those books which have more than 50 ratings

In [42]:
final_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61853 entries, 7 to 486790
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   isbn                 61853 non-null  object
 1   book_title           61853 non-null  object
 2   book_author          61853 non-null  object
 3   year_of_publication  61853 non-null  object
 4   publisher            61853 non-null  object
 5   image_url_s          61853 non-null  object
 6   image_url_m          61853 non-null  object
 7   image_url_l          61853 non-null  object
 8   user_id              61853 non-null  int64 
 9   book_rating          61853 non-null  int64 
 10  number_of_ratings    61853 non-null  int64 
dtypes: int64(3), object(8)
memory usage: 5.7+ MB


In [43]:
# removing those records where same user rated on same book for multiple times 
# note : it could be more perfectly done if we had a meta data , that tells us the latest updated rating on the same book by the same user , but unfortunately we dont have this  
final_ratings.drop_duplicates(['user_id','book_title'], inplace =True) 

In [44]:
final_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59850 entries, 7 to 486790
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   isbn                 59850 non-null  object
 1   book_title           59850 non-null  object
 2   book_author          59850 non-null  object
 3   year_of_publication  59850 non-null  object
 4   publisher            59850 non-null  object
 5   image_url_s          59850 non-null  object
 6   image_url_m          59850 non-null  object
 7   image_url_l          59850 non-null  object
 8   user_id              59850 non-null  int64 
 9   book_rating          59850 non-null  int64 
 10  number_of_ratings    59850 non-null  int64 
dtypes: int64(3), object(8)
memory usage: 5.5+ MB


## Top 50 Books

In [46]:
count = final_ratings[final_ratings['number_of_ratings']>50] #top 50 if that book got min 250 ratings
avg_ratings = count.groupby('book_title')['book_rating'].mean().reset_index().sort_values(by='book_rating', ascending=False)
avg_ratings.rename(columns = {'book_rating' : 'average_ratings'} , inplace =True)

In [47]:
avg_ratings.shape

(703, 2)

In [48]:
avg_ratings

,book_title,average_ratings
563,The Little Prince,5.115385
228,Harry Potter and the Sorcerer's Stone (Book 1),4.710526
4,84 Charing Cross Road,4.666667
227,Harry Potter and the Prisoner of Azkaban (Book 3),4.451128
225,Harry Potter and the Goblet of Fire (Book 4),4.444444
...,...,...
377,Pleading Guilty,0.539683
173,Exclusive,0.442308
691,Wild Animus,0.438017
305,Long After Midnight,0.433962


In [49]:
top_50 = avg_ratings.iloc[:50]
top_50 = top_50.merge(final_ratings , on='book_title')
top_50.drop_duplicates(['book_title'],inplace =True)

In [50]:
top_50.shape

(50, 12)

In [51]:
top_50.head(3)

,book_title,average_ratings,isbn,book_author,year_of_publication,publisher,image_url_s,image_url_m,image_url_l,user_id,book_rating,number_of_ratings
0,The Little Prince,5.115385,0156528207,Antoine de Saint-ExupÃ©ry,1968,Harcourt,http://images.amazon.com/images/P/0156528207.0...,http://images.amazon.com/images/P/0156528207.0...,http://images.amazon.com/images/P/0156528207.0...,278418,0,53
52,Harry Potter and the Sorcerer's Stone (Book 1),4.710526,0590353403,J. K. Rowling,1998,Scholastic,http://images.amazon.com/images/P/0590353403.0...,http://images.amazon.com/images/P/0590353403.0...,http://images.amazon.com/images/P/0590353403.0...,254,9,78
128,84 Charing Cross Road,4.666667,0860074382,Helene Hanff,0,Warner Books> C/o Little Br,http://images.amazon.com/images/P/0860074382.0...,http://images.amazon.com/images/P/0860074382.0...,http://images.amazon.com/images/P/0860074382.0...,11676,7,54


In [175]:
# If each URL needs to be split by ':' and you want the second part needed for heroku
f = [url.replace('http://', 'https://') for url in top_50['image_url_m']]


In [177]:
f[0]

'https://images.amazon.com/images/P/0156528207.01.MZZZZZZZ.jpg'

In [52]:
import pickle 
pickle.dump(top_50,open('top_50.pkl','wb'))

## above are Top 50 books

## Now for Recommendations ↓

In [55]:
final_pivot = final_ratings.pivot_table(columns = 'user_id' , index = 'book_title' , values = 'book_rating').replace(np.nan , 0) #can also use fillna(0) it's her choice 😗

In [56]:
final_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
book_title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_score = cosine_similarity(final_pivot)  # it's hold a eucladian dist. of every books of 888 dimentional space
similarity_score.shape

(742, 742)

In [58]:
similarity_score
# important note : similarity_score is a numpy array which doesnot hold the index name (here books name) so we need to pass the index of the book name (like book_[0] = 'attack on titan')

array([[1.        , 0.07624004, 0.        , ..., 0.09387814, 0.04480685,
        0.03286937],
       [0.07624004, 1.        , 0.24424725, ..., 0.07424784, 0.16365457,
        0.15050832],
       [0.        , 0.24424725, 1.        , ..., 0.0432679 , 0.04617844,
        0.10992264],
       ...,
       [0.09387814, 0.07424784, 0.0432679 , ..., 1.        , 0.07085128,
        0.03898126],
       [0.04480685, 0.16365457, 0.04617844, ..., 0.07085128, 1.        ,
        0.13208788],
       [0.03286937, 0.15050832, 0.10992264, ..., 0.03898126, 0.13208788,
        1.        ]])

In [149]:
def recommend_books(book_name):
    index = final_pivot.index.get_loc(book_name) # taking the index position
    similar_items = sorted(list(enumerate(similarity_score[index])) ,key = lambda x:x[1] ,reverse = True)[1:6]
   
    # Filter final_ratings to get rows where book_title is in final_pivot.index[i[0]] for i in some_iterable
    filtered_df = final_ratings[final_ratings['book_title'].isin([final_pivot.index[i[0]] for i in similar_items])]
    
    # Remove duplicates based on book_title, keeping the first occurrence
    filtered_df = filtered_df.drop_duplicates(['book_title']).drop(columns=['isbn','year_of_publication','user_id','publisher'])
        
    return filtered_df

In [151]:
recommend_books('Message in a Bottle')

,book_title,book_author,image_url_s,image_url_m,image_url_l,book_rating,number_of_ratings
11223,River's End,Nora Roberts,http://images.amazon.com/images/P/0515127833.0...,http://images.amazon.com/images/P/0515127833.0...,http://images.amazon.com/images/P/0515127833.0...,0,77
14593,Nights in Rodanthe,Nicholas Sparks,http://images.amazon.com/images/P/0446531332.0...,http://images.amazon.com/images/P/0446531332.0...,http://images.amazon.com/images/P/0446531332.0...,0,62
57906,The Mulberry Tree,Jude Deveraux,http://images.amazon.com/images/P/0743437640.0...,http://images.amazon.com/images/P/0743437640.0...,http://images.amazon.com/images/P/0743437640.0...,3,69
67893,A Walk to Remember,Nicholas Sparks,http://images.amazon.com/images/P/0446608955.0...,http://images.amazon.com/images/P/0446608955.0...,http://images.amazon.com/images/P/0446608955.0...,0,137
142824,Sanctuary,Nora Roberts,http://images.amazon.com/images/P/0515122734.0...,http://images.amazon.com/images/P/0515122734.0...,http://images.amazon.com/images/P/0515122734.0...,10,78


In [ ]:
## Converting final_pivot , similarity_score ,final_ratings to pkl for recommendations for web-in.

In [153]:

pickle.dump(final_pivot,open('final_pivot.pkl','wb'))
pickle.dump(similarity_score,open('similarity_score.pkl','wb'))
pickle.dump(final_ratings,open('final_ratings.pkl','wb'))
